# GitHubAgent runtime test

This notebook performs a runtime test of `doppiozero.agents.gh_deep_search.GitHubAgent`.
By default the notebook will use your real LLM and content layer when available. To force deterministic offline behavior set `USE_MOCKS = True` in the mocking cell.

Run the cells in order. The notebook saves a single `agent_test_output.json` artifact when complete.

In [1]:
import types, json
from dotenv import load_dotenv
from pathlib import Path
from pprint import pprint
load_dotenv()
from doppiozero.agents.gh_deep_search import GitHubAgent

In [2]:
clarifying_answers = '''Q: What is the main goal?
A: Find recent authentication failure discussions across repos.

Q: Are there specific repos to focus on?
A: doppiozero and financial-planning.
'''

clarifying_qa = Path('clarifying_answers.txt').resolve()
clarifying_qa.write_text(clarifying_answers, encoding='utf-8')
print('Wrote clarifying answers to', clarifying_qa)

Wrote clarifying answers to /Users/romanofoti/romanofoti/doppiozero/clarifying_answers.txt


## Run the agent

Instantiate `GitHubAgent` with lightweight options and run it.

In [3]:
options = {
    'collection': None,
    'limit': 3,
    'max_depth': 1,
    'editor_file': None,
    'clarifying_qa': clarifying_qa,
    'search_modes': ['semantic'],
    'cache_path': None,
    'models': {'fast': 'default', 'reasoning': 'default', 'embed': 'default'},
    'parallel': False,
    'verbose': True,
}
agent = GitHubAgent('What are the recent discussions about authentication failures?', options)
result = agent.run()
print('=== AGENT RUN RESULT ===')
pprint(result)
print('=== SHARED STATE ===')
pprint(agent.shared)

2025-09-05 19:53:39,020 - doppiozero.agents.gh_deep_search - gh_deep_search.py  - run          - INFO     - === GITHUB CONVERSATIONS RESEARCH AGENT ===
2025-09-05 19:53:39,021 - doppiozero.agents.gh_deep_search - gh_deep_search.py  - run          - INFO     - Request: What are the recent discussions about authentication failures?
2025-09-05 19:53:39,021 - doppiozero.agents.gh_deep_search - gh_deep_search.py  - run          - INFO     - Collection: None
2025-09-05 19:53:39,021 - doppiozero.agents.gh_deep_search - gh_deep_search.py  - run          - INFO     - Max results per search: 3
2025-09-05 19:53:39,022 - doppiozero.agents.gh_deep_search - gh_deep_search.py  - run          - INFO     - Max deep research iterations: 1
2025-09-05 19:53:39,022 - doppiozero.agents.gh_deep_search - gh_deep_search.py  - run          - INFO     - Fast model: default
2025-09-05 19:53:39,023 - doppiozero.agents.gh_deep_search - gh_deep_search.py  - run          - INFO     - Reasoning model: default
2025-09-

=== AGENT RUN RESULT ===
{'claims_verified': 0,
 'draft': "{'id': 'chatcmpl-CCUvx7KKKEDI2VKBU007QJHgEO904', 'choices': "
          "[{'finish_reason': 'length', 'index': 0, 'logprobs': None, "
          "'message': {'content': '', 'refusal': None, 'role': 'assistant', "
          "'annotations': [], 'audio': None, 'function_call': None, "
          "'tool_calls': None}, 'content_filter_results': {}}], 'created': "
          "1757094881, 'model': 'gpt-5-mini-2025-08-07', 'object': "
          "'chat.completion', 'service_tier': None, 'system_fingerprint': "
          "None, 'usage': {'completion_tokens': 1024, 'prompt_tokens': 153, "
          "'total_tokens': 1177, 'completion_tokens_details': "
          "{'accepted_prediction_tokens': 0, 'audio_tokens': 0, "
          "'reasoning_tokens': 1024, 'rejected_prediction_tokens': 0}, "
          "'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, "
          "'prompt_filter_results': [{'prompt_index': 0, "
          "'conte

2025-09-04 15:46:20,239 - doppiozero.agents.gh_deep_search - gh_deep_search.py  - run          - INFO     - Max deep research iterations: 1


2025-09-04 15:46:20,239 - doppiozero.agents.gh_deep_search - gh_deep_search.py  - run          - INFO     - Fast model: default


2025-09-04 15:46:20,239 - doppiozero.agents.gh_deep_search - gh_deep_search.py  - run          - INFO     - Reasoning model: default


...

2025-09-04 15:46:20,250 - doppiozero.nodes.reporter - reporter.py        - post         - INFO     - 1. Claim 1


2025-09-04 15:46:20,250 - doppiozero.nodes.reporter - reporter.py        - post         - INFO     - 2. Claim 2


2025-09-04 15:46:20,251 - doppiozero.nodes.reporter - reporter.py        - post         - INFO     - 

✓ Research complete! Total conversations analyzed: 13, 2 claims verified (0 supported, 2 unsupported)


2025-09-04 15:46:20,251 - doppiozero.nodes.end - end.py             - exec         - INFO     - End node: terminating workflow and returning results.


=== AGENT RUN RESULT ===
{'claims_verified': 0,
 'draft': '{}',
 'num_conversations': 13,
 'unsupported_claims': ['Claim 1', 'Claim 2']}
=== SHARED STATE ===
{'cache_path': None,
 'claim_verification': {'details': [{'claim': 'Claim 1',
                                     'evidence': [{'score': 1.0,
                                                   'snippet': 'Summary of '
                                                              'Claim 1 #0',
                                                   'source': 'https://example.com/convo/0'},
                                                  {'score': 0.9,
                                                   'snippet': 'Summary of '
                                                              'Claim 1 #1',
                                                   'source': 'https://example.com/convo/1'},
                                                  {'score': 0.8,
                                                   'snippet': 'Summary of '
   

In [4]:
# Simple runtime checks (no external test files created)
assert isinstance(result, dict), 'Agent result should be a dict'
# The agent returns a structured final report placed in shared['final_report']
assert 'num_conversations' in result, 'Result missing num_conversations key'
print('Basic assertions passed: result appears to be a final report dict')

Basic assertions passed: result appears to be a final report dict


In [5]:
# Save the result artifact for inspection
import json
out_path = Path('agent_test_output.json').resolve()
with open(out_path, 'w', encoding='utf-8') as f:
    json.dump(result, f, ensure_ascii=False, indent=2)
print('Saved agent output to', out_path)

Saved agent output to /Users/romanofoti/romanofoti/doppiozero/agent_test_output.json
